In [ ]:
from dotenv import load_dotenv
import pandas as pd
import pydeck as pdk

load_dotenv()

DATA_URL = "https://raw.githubusercontent.com/ajduberstein/geo_datasets/master/housing.csv"
df = pd.read_csv(DATA_URL)

view = pdk.data_utils.compute_view(df[["lng", "lat"]])
view.pitch = 75
view.bearing = 60

column_layer = pdk.Layer(
    "ColumnLayer",
    data=df,
    get_position=["lng", "lat"],
    get_elevation="price_per_unit_area",
    elevation_scale=100,
    radius=50,
    get_fill_color=["mrt_distance * 10", "mrt_distance", "mrt_distance * 10", 140],
    pickable=True,
    auto_highlight=True,
    diskResolution= 4
)


tooltip = {
    "html": "<b>{mrt_distance}</b> meters away from an MRT station, costs <b>{price_per_unit_area}</b> NTD/sqm",
    "style": {"background": "grey", "color": "white", "font-family": '"Helvetica Neue", Arial', "z-index": "10000"},
}

r = pdk.Deck(
    column_layer,
    initial_view_state=view,
    tooltip=tooltip,
    map_provider="mapbox",
    map_style=pdk.map_styles.CARTO_DARK_NO_LABELS,
)

# r.to_html("column_layer.html")
r

In [ ]:
df

In [ ]:
import pydeck

DATA_URL = "https://raw.githubusercontent.com/visgl/deck.gl-data/master/examples/geojson/vancouver-blocks.json"
LAND_COVER = [[[-123.0, 49.196], [-123.0, 49.324], [-123.306, 49.324], [-123.306, 49.196]]]

coordinates = [[-123.0, 49.196], [-123.0, 49.324], [-123.306, 49.324], [-123.306, 49.196]]
names = ['Location A', 'Location B', 'Location C', 'Location D']

locations = [{'name': name, 'coordinates': coord} for name, coord in zip(names, coordinates)]


INITIAL_VIEW_STATE = pydeck.ViewState(
  latitude=49.254,
  longitude=-123.13,
  zoom=11,
  max_zoom=16,
  pitch=45,
  bearing=0
)

polygon = pydeck.Layer(
    'PolygonLayer',
    LAND_COVER,
    stroked=False,
    get_polygon='-',
    get_fill_color=[255, 0, 0, 20]
)

labels = pydeck.Layer(
    'TextLayer',
    locations,
    get_position="coordinates",
    get_text="name",
    stroked=False,
    get_fill_color=[255, 0, 0, 20],
    get_polygon='-',
)

geojson = pydeck.Layer(
    'GeoJsonLayer',
    DATA_URL,
    opacity=0.8,
    stroked=False,
    filled=True,
    extruded=True,
    wireframe=True,
    get_elevation='properties.valuePerSqm / 20',
    get_fill_color='[255, 255, properties.growth * 255]',
    get_line_color=[255, 255, 255],
    pickable=True
)

r = pydeck.Deck(
    layers=[labels, polygon],
    initial_view_state=INITIAL_VIEW_STATE)

r.to_html()

In [ ]:
import os
import sys
import geopandas as gpd
sys.path.append('../src')

from infrastructure.postgres_db import PostgresDB
db_name = os.environ.get("DB_NAME")
db_user = os.environ.get("DB_USER")
db_password = os.environ.get("DB_PASSWORD")


db = PostgresDB(db_name=db_name, db_user=db_user, db_password=db_password)


In [ ]:
comunas.explore()

In [ ]:
comunas.geom.centroid.explore()

In [ ]:
comunas.geom.centroid.coordinates

In [ ]:
comunas = db.run_gpd_query("SELECT * FROM comuna")

# Function to extract coordinates
def get_polygon_coordinates(geom):
    if geom.type == 'Polygon':
        return [[(x, y) for x, y in geom.exterior.coords]]
    elif geom.type == 'MultiPolygon':
        return [[(x, y) for x, y in polygon.exterior.coords] for polygon in geom.geoms]
    else:
        return None

comunas = comunas.to_crs(epsg=4326)

comunas = comunas.assign(
    lat = comunas.geom.centroid.y,
    lon = comunas.geom.centroid.x,
    coordinates = comunas.geom.apply(lambda geom: [list(coord) for coord in geom.geoms[0].exterior.coords]),
    coordinates2 = comunas.geom.apply(get_polygon_coordinates),
    score=1000
)

# view = pdk.data_utils.compute_view(comunas[["lon", "lat"]])
# view.pitch = 75
# view.bearing = 60

column_layer = pdk.Layer(
    "ColumnLayer",
    data=comunas.head(10),
    get_position=["lon", "lat"],
    get_elevation="score",
    elevation_scale=100,
    radius=5000,
    get_fill_color=[255, 0, 0, 255],
    pickable=True,
    auto_highlight=True,
    diskResolution= 4
)

polygon = pydeck.Layer(
    'PolygonLayer',
    data=comunas.head(5),
    get_polygon="coordinates",
    stroked=False,
    # get_polygon='-',
    get_fill_color=[255, 0, 0, 255],
    line_width_min_pixels=1,
)

# tooltip = {
#     "html": "<b>{mrt_distance}</b> meters away from an MRT station, costs <b>{price_per_unit_area}</b> NTD/sqm",
#     "style": {"background": "grey", "color": "white", "font-family": '"Helvetica Neue", Arial', "z-index": "10000"},
# }

r = pdk.Deck(
    layers=[column_layer, polygon],
    # initial_view_state=view,
    # tooltip=tooltip,
    map_provider="mapbox",
    map_style=pdk.map_styles.CARTO_LIGHT,
)

# r.to_html("column_layer.html")
r.show()

In [ ]:
column_layer

In [ ]:
list(comunas.geom.iloc[0].geoms[0].exterior.coords)

In [ ]:
comunas

In [ ]:
comunas = comunas.to_crs(epsg=4326).assign(
    lat = comunas.geom.centroid.y,
    lon = comunas.geom.centroid.x,
    coordinates = comunas.geom.apply(lambda geom: [list(coord) for coord in geom.geoms[0].exterior.coords]),
    score=10000
)

In [ ]:
polygon = pydeck.Layer(
    'PolygonLayer',
    data=comunas.head(5),
    get_polygon="coordinates",
    stroked=False,
    # get_polygon='-',
    get_fill_color=[255, 0, 0, 20],
    line_width_min_pixels=1,
)

polygon

In [ ]:
comunas

In [ ]:
len(comunas.polygon.iloc[0][0])

In [ ]:
comunas.geom.apply(lambda geom: list([a.coords for a in geom.geoms][0]))

In [ ]:
len(list([a for a in comunas.geom.iloc[0].geoms][0].exterior.coords))

In [ ]:
comunas_polycomunas

In [ ]:
comunas.geometry.

In [ ]:
comunas.head(5).explore()

In [ ]:
comunas.head(10)

In [ ]:
comunas

In [ ]:
df

In [ ]:
# comunas.to_crs(epsg=3857).geom.centroid.x
comunas.to_crs(epsg=4326).geom.centroid.x



In [ ]:
comunas.head(10).explore()

In [ ]:
deck is None

# With elections data

In [ ]:
df

In [ ]:
import pandas as pd

# Sample data
data = {
    'Location': ['Location A', 'Location B', 'Location C'],
    'Latitude': [37.7749, 34.0522, 40.7128],
    'Longitude': [-122.4194, -118.2437, -74.0060],
    'Value': [10, 20, 30]
}
df = pd.DataFrame(data)

import folium
import plotly.express as px
import plotly.graph_objs as go
from folium.plugins import MarkerCluster
import base64
from io import BytesIO

# Create a folium map
m = folium.Map(location=[39.50, -98.35], zoom_start=4)

# Create a marker cluster
marker_cluster = MarkerCluster().add_to(m)

# Function to create a Plotly bar chart and return it as HTML
def create_bar_chart(location, value):
    fig = px.bar(x=['Value'], y=[value], labels={'x': 'Metric', 'y': 'Value'}, title=location)
    fig.update_layout(margin=dict(l=0, r=0, t=30, b=0))
    buf = BytesIO()
    fig.write_html(buf, include_plotlyjs='cdn')
    return buf.getvalue()

# Add markers with bar charts to the map
for idx, row in df.iterrows():
    chart_html = create_bar_chart(row['Location'], row['Value'])
    encoded = base64.b64encode(chart_html.encode('utf-8')).decode('utf-8')
    iframe = folium.IFrame(html=f'<iframe src="data:text/html;base64,{encoded}" width="400" height="300"></iframe>', width=400, height=300)
    popup = folium.Popup(iframe, max_width=400)
    folium.Marker(location=[row['Latitude'], row['Longitude']], popup=popup).add_to(marker_cluster)

# Save the map to an HTML file
m.save('map_with_bar_charts.html')